# Helper Functions

This notebook contains some functions that are used frequently throughout this project. They are explained here. Copies of these functions are available in `modules/helper.py` (with the exception of the dataset class) so that they can be imported in the different notebooks.

In [28]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn.functional as F

from PIL import Image
from torchvision import models
from IPython.display import clear_output
plt.rcParams['figure.figsize'] = (20.0, 10.0)

In [11]:
## Load model and data
model = models.googlenet(pretrained=True)
model.eval()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]   

preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

data_loader = torch.utils.data.DataLoader(
    ImageNetSubset("data/ImageNet_subset/dev_dataset.csv", "data/ImageNet_subset/images/", transform=preprocess))

## Data

For the data we use a subset of the ImageNet [1] dataset. It was part of the competition [NIPS 2017: Adversarial Learning Development Set](https://www.kaggle.com/google-brain/nips-2017-adversarial-learning-development-set#categories.csv) [2] on Kaggle and is available [here](http://www.kaggle.com).

This class is stored in the file `modules/dataset.py`.

In [3]:
class ImageNetSubset(Dataset):
    '''Imports subset of the ImageNet dataset from the Kaggle competion'''
    
    def __init__(self, csv_file, root_dir, transform=None):
        '''
        Args:
        csv_file (string)              -- Path to the csv file with metadata like labels and fileId.
        root_dir (string)              -- Directory with all the images.
        transform (callable, optional) -- Optional transform to be applied on a sample.
        '''

        self.images_meta = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
               
    def __len__(self):
        return len(self.images_meta)
    
    def __getitem__(self, idx):     
        image_path = self.root_dir
        image_name = self.images_meta["ImageId"][idx]
        label = self.images_meta["TrueLabel"][idx]
        
        ## Load image
        image = Image.open(image_path + image_name + ".png")
        
        if self.transform is not None:
            image = self.transform(image)
            
        ## Format label. Labels in dataset are 1 indexed but 0 indexed in model. Make all 0 indexed.
        label = torch.tensor(label-1, dtype=torch.long)
        
        ## Move data to cuda if available
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        image = image.to(device)
        label = label.to(device)
        
        
        return image, label

In [4]:
def idx_to_name(idx):
    '''
    Converts the output class index from the googleNet to the respective name.
    
    Input:
    idx  -- Class index as integer
    
    Returns:
    name -- Class names corresponding to idx as string
    '''
    
    ## Load dictionary from file    
    names = pd.read_csv("./data/ImageNet_subset/categories.csv")
    
    ## Retrieve class name for idx
    name = names.iloc[idx]["CategoryName"]
    
    return name

In [5]:
def show_tensor_image(tensor):
    '''
    De-normalizes an image as a tensor and converts it back into an 8bit image object.
    
    Inputs:
    tensor -- PyTorch tensor of shape (1, 3, 224, 224)
    
    Returns:
    image  -- De-normalized image object
    '''
    
    ## Detach computation graph and remove batch dimension
    tensor = tensor.detach().clone()    
    tensor.squeeze_()
    
    ## De-normalize tensor image
    invert_preprocess = transforms.Compose([
        transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225], std=[1/0.229, 1/0.224, 1/0.225]),
    ])
      
    image = invert_preprocess(tensor)      
    image = np.array(image.detach())
    
    ## Rescale to range 0-255 and convert datatype into 8bit
    image = image * 255    
    image = np.uint8(image)
    
    ## Swap axes to get the expected shape (224, 224, 3)
    image = np.swapaxes(image, 0, 2)
    
    ## Rotate and flip the image, then convert to image object
    image = np.flipud(np.rot90(image))
    
    image = Image.fromarray(image)
        
    return image

## Predict

We use the trained googleNet [3]. It is available through the `Torchvision` library. It expects the data to be 3 channel RGB with a size of at least 224. The data has to be scaled into the range $[0, 1]$ and then centered, see [here](https://pytorch.org/hub/pytorch_vision_googlenet/).

In [26]:
def predict(model, image, target_label, return_grad=False):
    '''
    Predicts the class of the given image and compares the prediction with the provided label.
    
    Inputs:
    model             -- net
    image             -- Input image as tensor of shape (1, 3, 224, 224)
    target_label      -- Target label as tensor of shape (1)
    return_grad       -- Returns gradient if set True
    
    Returns:
    predicted_classes -- Numpy array of top 5 predicted class indices
    confidences       -- Numpy array of top 5 confidences in descending order
    gradient          -- None if return_grad=False. Otherwise the gradient from the prediction
                         as a tensor of shape ().
    '''      
     
    if return_grad == True:
        image.requires_grad=True
        prediction = model(image)
               
        # Zero gradients
        model.zero_grad()

        # Calculate loss using the class index for pandas and get gradient
        loss = F.nll_loss(prediction, target_label)
        loss.backward()
        gradient = image.grad.data
        
    else:           
        gradient = None
        with torch.no_grad():
            prediction = model(image)  

    # Get class index and confidence for prediction 
    prediction = torch.nn.functional.softmax(prediction[0].cpu().detach(), dim=0).numpy()
   
    # Get top 5 class indices
    predicted_classes = prediction.argsort()[-5:][::-1]
        
    # Get largest confidences
    confidences = prediction[predicted_classes]
    
    return predicted_classes, confidences, gradient

## Visualize Images

In [1]:
def summarize_attack(image_clean, image_adv, conf_clean, conf_adv, label_clean, label_adv, label_target, idx,
                    folder=None):
    '''
    Summarizes attack by printing info and displaying the image along with the adversary and the isolated
    perturbance. Saves image to the folder
    
    Inputs:
    image_clean     -- Clean image as tensor of shape (1, 1, 28, 28)
    image_adv       -- Adversarial image as tensor of shape (1, 1, 28, 28)
    conf_clean      -- Confidence for the clean image
    conf_adv        -- Confidence for the adversarial image
    label_clean     -- Predicted label from the clean image
    label_adv       -- Predicted label from the adversarial image
    label_target    -- Target label as tensor of shape (1)
    idx             -- Sample index used for filename of plot export
    folder          -- If not None folder to which the image is saved.
    '''
   
    ## Get label names from index
    name_target = idx_to_name(label_target.detach().numpy()[0])
    name_clean = idx_to_name(label_clean)
    name_adv = idx_to_name(label_adv)
    
    ## Isloate perturbance
    perturbance = image_adv - image_clean
    
    ## Text
    print("\t\t\tClean image\t Adversarial image\n")    
    print("Actual class: \t\t{}\t\t\t{}".format(name_target, name_target ))
    print("Predicted class: \t{}\t\t\t{}".format(name_clean, name_adv ))
    print("Confidence: \t\t{:.2f}%\t\t\t\t{:.2f}%\n".format(conf_clean*100, conf_adv*100))
    
    ## Plots
    f = plt.figure(figsize=(20,20))
    
    ax = f.add_subplot(221)
    ax2 = f.add_subplot(222)
    ax3 = f.add_subplot(223)
    
    im1 = show_tensor_image(image_clean)
    im2 = show_tensor_image(perturbance)
    im3 = show_tensor_image(image_adv)
    
    ax.imshow(im1)
    ax.set_title("Clean example", fontsize=25)
    ax.axis('off')
    ax2.imshow(im2)
    ax2.set_title("Perturbance", fontsize=25)
    ax2.axis('off')
    ax3.imshow(im3)
    ax3.set_title("Adversarial example", fontsize=25)
    ax3.axis('off')
    
    ## Save figure
    if folder is not None:
        f.tight_layout()
        f.savefig("plots/" + str(folder) + "/"+ str(folder) +"-sample_" + str(int(idx)) + "_pair.png")